In [ ]:
%cd ..

In [ ]:
import datasets
import json
import os
from glob import glob

In [ ]:
pairs = json.load(open('data/TextHard/raw_splits.json', 'r'))
len(pairs)

In [ ]:
train = datasets.Dataset.from_list(pairs["train"])
test = datasets.Dataset.from_list(pairs["test"])

train

In [ ]:
test

In [ ]:
test[0]

In [ ]:
train[100]

In [ ]:
files = glob("data/batches_cogito*/*.json")
loaded_files = [json.load(open(f, 'r')) for f in files]
len(loaded_files)

In [ ]:
j = [sublist for sublist in loaded_files if isinstance(sublist, dict)]
len(j)

In [ ]:
j[10]

In [ ]:
len([i for i in j if "data" in i and isinstance(i["data"], list)])

In [ ]:
merged1 = [i for sublist in loaded_files if isinstance(sublist, list) for i in sublist]
merged2 = [
    i
    for sublist in loaded_files
    if isinstance(sublist, dict)
    and "data" in sublist
    and isinstance(sublist["data"], list)
    for i in sublist["data"] if isinstance(i, dict)
]
len(merged1), len(merged2)

In [ ]:
total = merged1 + merged2
len(total)

In [ ]:
total[0]

In [ ]:
import pandas as pd

df = pd.DataFrame(total)
df.sample(10)

In [ ]:
df.isna().mean()

In [ ]:
df[~df["label"].isna()]

In [ ]:
df.columns

In [ ]:
print(df.shape)
df = df.dropna(subset=["sentence"])
df.shape

In [ ]:
df.columns[df.columns.str.lower().str.startswith("label")]

In [ ]:
df.columns[df.columns.str.lower().str.startswith("not")]

In [ ]:
labels = list(df.columns[df.columns.str.lower().str.startswith("label")])
no_labels = list(df.columns[df.columns.str.lower().str.startswith("not")])

def choose_non_null_label(row):
	for label in labels:
		if isinstance(row[label], list) and len(row[label]) > 0:
			return row[label]
		elif isinstance(row[label], str) and len(row[label]) > 0:
			return [row[label]]
	return []

def choose_non_null_not_label(row):
	for label in no_labels:
		if isinstance(row[label], list) and len(row[label]) > 0:
			return row[label]
		elif isinstance(row[label], str) and len(row[label]) > 0:
			return [row[label]]
	return []

df['labels'] = df.apply(choose_non_null_label, axis=1)
df['not_labels'] = df.apply(choose_non_null_not_label, axis=1)
df = df[['sentence', 'labels', 'not_labels']]

In [ ]:
df.sample(10)

In [ ]:
df["sentence"].value_counts().head(20)

In [ ]:
df_clean = df.groupby("sentence")[['labels', 'not_labels']].apply(lambda x: pd.Series({
    "labels": list(set(sum(x['labels'].tolist(), start=[]))),
	"not_labels": list(set(sum(x['not_labels'].tolist(), start=[])))
})).reset_index()

df_clean.shape, df.shape

In [ ]:
df_clean.sample(10)

In [ ]:
df_clean["labels"].apply(len).describe()

In [ ]:
df_clean["not_labels"].apply(len).describe()

In [ ]:
df_clean[df_clean["not_labels"].apply(len) > 10]

In [ ]:
raw = df_clean.apply(lambda x: set(x['labels']) & set(x['not_labels']), axis=1)
raw

In [ ]:
raw.apply(len).describe()

In [ ]:
df_clean[raw.apply(len) > 0]

In [ ]:
df_clean = df_clean.apply(lambda x: {
    "sentence": x['sentence'],
	"labels": list(set(x['labels']) - set(x['not_labels'])),
	"not_labels": list(set(x['not_labels']) - set(x['labels']))
}, axis=1, result_type='expand')

df_clean.sample(10)

In [ ]:
df_clean["sentence"].apply(lambda x: x.split()).apply(len).describe()

In [ ]:
df_clean["sentence"].apply(len).describe()

In [ ]:
from collections import Counter
import re

# Extract all words from sentences and count their frequency
all_words = []
stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'his', 'her', 'its', 'our', 'their'}

for sentence in df_clean['sentence']:
	# Convert to lowercase and extract words (remove punctuation)
	words = re.findall(r'\b[a-zA-Z]+\b', sentence.lower())
	# Filter out common stopwords
	words = [word for word in words if word not in stopwords]
	all_words.extend(words)


# Count word frequencies
word_counts = Counter(all_words)

# Display most frequent words
print("Top 20 most frequent words:")
for word, count in word_counts.most_common(20):
	print(f"{word}: {count}")

In [ ]:
pairs_merged = pairs["train"] + pairs["test"]
len(pairs_merged)

In [ ]:
pairs_df = pd.DataFrame(pairs_merged)

pairs_df.sample(10)

In [ ]:
pairs_df["text"].value_counts().head(10)

In [ ]:
pairs_cleaned = pairs_df.groupby("text")[["labels"]].apply(
    lambda x: pd.Series(
        {
            "labels": list(set(sum(x["labels"].tolist(), start=[]))),
        }
    )
).reset_index()

pairs_cleaned.shape, pairs_df.shape

In [ ]:
pairs_cleaned["labels"].apply(len).describe()

In [ ]:
from collections import Counter
import re

# Extract all words from sentences and count their frequency
all_words = []
stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'his', 'her', 'its', 'our', 'their'}

for sentence in pairs_cleaned['text']:
	# Convert to lowercase and extract words (remove punctuation)
	words = re.findall(r'\b[a-zA-Z]+\b', sentence.lower())
	# Filter out common stopwords
	words = [word for word in words if word not in stopwords]
	all_words.extend(words)


# Count word frequencies
word_counts = Counter(all_words)

# Display most frequent words
print("Top 20 most frequent words:")
for word, count in word_counts.most_common(20):
	print(f"{word}: {count}")

In [ ]:
df_clean.shape, pairs_cleaned.shape

In [ ]:
df_clean.sample(10)

In [ ]:
df_clean = df_clean.rename(columns={"sentence": "text"})
df_clean = df_clean[(df_clean["labels"].apply(len) < 20) & (df_clean["not_labels"].apply(len) < 20)]
df_clean = df_clean.reset_index(drop=True)

In [ ]:
ds_triplet = datasets.Dataset.from_pandas(df_clean.rename(columns={"sentence": "text"}))
# ds_triplet = ds_triplet.rename_column("sentence", "text")

ds_couplet = datasets.Dataset.from_pandas(pairs_cleaned)

In [ ]:
ds_triplet

In [ ]:
ds_couplet

In [ ]:
ds = datasets.DatasetDict({
	"triplet": ds_triplet,
	"couplet": ds_couplet
})

ds

In [ ]:
pairs_cleaned.head()

In [ ]:
from collections import Counter
import numpy as np

# Flatten all labels to count frequencies
all_labels = sum(pairs_cleaned['labels'].tolist(), start=[])

# Count label frequencies
label_counts = Counter(all_labels)
total_labels = len(all_labels)

# Define rare labels (bottom 0.2% by frequency)
sorted_labels = sorted(label_counts.items(), key=lambda x: x[1])
rare_threshold_idx = int(len(sorted_labels) * 0.2)  # 0.2%
rare_labels = set([label for label, count in sorted_labels[:rare_threshold_idx]])

print(f"Total unique labels: {len(label_counts)}")
print(f"Rare labels (bottom 0.2%): {len(rare_labels)}")
print(f"Rare labels: {list(rare_labels)[:10]}...")  # Show first 10

# Vectorized operation to identify rows with rare labels
has_rare_labels = pairs_cleaned['labels'].apply(lambda labels_list: any(label in rare_labels for label in labels_list))
test_indices = has_rare_labels[has_rare_labels].index.tolist()

# Create test set with rare labels only - vectorized filtering
test_mask = pairs_cleaned.index.isin(test_indices)
test_subset = pairs_cleaned[test_mask].copy()

# Filter labels to keep only rare ones using vectorized operation
test_subset['labels'] = test_subset['labels'].apply(lambda labels_list: [label for label in labels_list if label in rare_labels])

# Remove rows where no rare labels remain after filtering
test_df = test_subset[test_subset['labels'].apply(len) > 0].reset_index(drop=True)

# Create train set (all rows not in test)
train_df = pairs_cleaned[~test_mask].reset_index(drop=True)

print(f"\nTrain set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"Test set labels sample: {test_df['labels'].iloc[0] if len(test_df) > 0 else 'No test data'}")


In [ ]:
test_df.sample(10)

In [ ]:
ds_couplet = datasets.DatasetDict({
	"train": datasets.Dataset.from_pandas(train_df),
	"test": datasets.Dataset.from_pandas(test_df)
})

ds_couplet

In [ ]:
from collections import Counter
import numpy as np

# Flatten all labels to count frequencies
all_labels = sum(df_clean['labels'].tolist(), start=[])

# Count label frequencies
label_counts = Counter(all_labels)
total_labels = len(all_labels)

# Define rare labels (bottom 0.2% by frequency)
sorted_labels = sorted(label_counts.items(), key=lambda x: x[1])
rare_threshold_idx = int(len(sorted_labels) * 0.2)  # 0.2%
rare_labels = set([label for label, count in sorted_labels[:rare_threshold_idx]])

print(f"Total unique labels: {len(label_counts)}")
print(f"Rare labels (bottom 0.2%): {len(rare_labels)}")
print(f"Rare labels: {list(rare_labels)[:10]}...")  # Show first 10

# Vectorized operation to identify rows with rare labels
has_rare_labels = df_clean['labels'].apply(lambda labels_list: any(label in rare_labels for label in labels_list))
test_indices = has_rare_labels[has_rare_labels].index.tolist()

# Create test set with rare labels only - vectorized filtering
test_mask = df_clean.index.isin(test_indices)
test_subset = df_clean[test_mask].copy()

# Filter labels to keep only rare ones using vectorized operation
test_subset['labels'] = test_subset['labels'].apply(lambda labels_list: [label for label in labels_list if label in rare_labels])

# Remove rows where no rare labels remain after filtering
test_df = test_subset[test_subset['labels'].apply(len) > 0].reset_index(drop=True)

# Create train set (all rows not in test)
train_df = df_clean[~test_mask].reset_index(drop=True)

print(f"\nTrain set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"Test set labels sample: {test_df['labels'].iloc[0] if len(test_df) > 0 else 'No test data'}")


In [ ]:
ds_triplet = datasets.DatasetDict({
	"train": datasets.Dataset.from_pandas(train_df),
	"test": datasets.Dataset.from_pandas(test_df)
})

ds_triplet

In [ ]:
ds = datasets.DatasetDict({
	"triplet": ds_triplet,
	"couplet": ds_couplet
})

ds

In [ ]:
# ds.push_to_hub("alexneakameni/ZSHOT-HARDSET", commit_message="alexneakameni: Initial commit of ZSHOT-HARDSET dataset")

In [ ]:
# Push each subset under a different config name
ds["triplet"].push_to_hub(
    "alexneakameni/ZSHOT-HARDSET",
    # split="triplet",
    config_name="triplet",
    commit_message="alexneakameni: Upload triplet subset -- updated",
)
ds["couplet"].push_to_hub(
    "alexneakameni/ZSHOT-HARDSET",
    # split="couplet",
    config_name="couplet",
    commit_message="alexneakameni: Upload couplet subset -- updated",
)